In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import folium

In [ ]:
def make_con():
    user = "surf_app"
    pwd = "password1"
    host = "localhost"
    db = "surfing_project"
    con_string = f"postgresql+psycopg2://{user}:{pwd}@{host}/{db}"
    engine = create_engine(con_string)
    return engine


In [ ]:
engine = make_con()

sql = "SELECT * FROM public.\"BouyMeta\";"
bouy_meta = pd.read_sql(sql, con=engine)

    # Bouys in the East
bouy_meta = bouy_meta[bouy_meta["lon"].between(-84, -55)
                          & bouy_meta["lat"].between(23, 47)]
    # But not the great lakes
bouy_meta_GL = bouy_meta[bouy_meta["lon"].between(-84, -72) & bouy_meta["lat"].between(41.2, 48)]

bouy_meta = pd.concat([bouy_meta, bouy_meta_GL]) 
bouy_meta = bouy_meta.drop_duplicates(keep=False)


    # marking metadata
m = folium.Map(location=[bouy_meta['lat'].iloc[0],
                            bouy_meta['lon'].iloc[0]], zoom_start=5)

for index, row in bouy_meta.iterrows():
    folium.Marker(location=[row['lat'], row['lon']],popup=row['name']).add_to(m)

In [ ]:
m

In [ ]:
engine = make_con()

bouy_meta = pd.read_xml("https://www.ndbc.noaa.gov/activestations.xml")

sql = "SELECT * FROM public.\"BouyMeta\";"

bouy_loaded = pd.read_sql(sql, con=engine)

existing_bouys = bouy_loaded['id'].values

bouy_meta = bouy_meta[~bouy_meta['id'].isin(existing_bouys)]

bouy_meta.to_sql("BouyMeta",
                 con=engine,
                 if_exists='append',
                 index='False')

In [ ]:
bouy_meta

In [ ]:
bouy_meta[bouy_meta["lon"].between(-84,-55) 
          & bouy_meta["lat"].between(23,47)]

In [ ]:
bouy_meta = pd.read_xml("https://www.ndbc.noaa.gov/activestations.xml")

bouy_meta = bouy_meta[bouy_meta["lon"].between(-84,-55) 
          & bouy_meta["lat"].between(23,47)]

bouy_meta.to_sql("BouyMetaNorthEast",
                     con=engine,
                     if_exists='replace',
                     index='False')

In [ ]:
bouy_meta=bouy_meta[bouy_meta["lon"].between(-84,-55) 
                    & bouy_meta["lat"].between(23,47)]

m = folium.Map(location=[bouy_meta['lat'].iloc[0], bouy_meta['lon'].iloc[0]], zoom_start=5)

# Add markers for each location
for index, row in bouy_meta.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name']
    ).add_to(m)

# Display the map
m

In [ ]:
bouy_meta

In [ ]:
m

In [ ]:
from io import BytesIO
import io
import base64
from PIL import Image

In [ ]:
buffer = BytesIO()
img = m._to_png(5)
img = Image.open(io.BytesIO(img))
img.save(buffer, format="PNG")
image_data = base64.b64encode(buffer.getvalue())

# Convert the image to Markdown to preview it within Dagster
md_content = f"![img](data:image/png;base64,{image_data.decode()})"

In [ ]:
img.save(buffer, format="PNG")
image_data = base64.b64encode(buffer.getvalue())

# Convert the image to Markdown to preview it within Dagster
md_content = f"![img](data:image/png;base64,{image_data.decode()})"

In [ ]:
md_content